# Dynamic Programming Project 2023
---
This notebook solves the project presented in the `model.py` file

ChatGPT has the following answer to the question:
- *How would you extend a consumption savings model in a dynamic programming course project?*

"Extending a consumption savings model in a dynamic programming course project can involve adding additional features to the basic model or modifying the assumptions of the model to make it more realistic. Here are a few ideas for extending a consumption savings model in a dynamic programming project:

**Add constraints**: One way to make the consumption savings model more realistic is to add constraints to the optimization problem. For example, you could add a borrowing constraint to the model to limit the amount of debt that households can take on. This would affect their consumption and savings decisions, as they would have to balance their desire to consume with their ability to borrow.

**Incorporate uncertainty**: Another way to extend the model is to incorporate uncertainty into the decision-making process. You could add a stochastic element to the model, such as a random income shock, to simulate the uncertainty that households face in the real world. This would require you to use dynamic programming methods that are suited to stochastic optimization problems.

**Consider heterogeneous agents**: A more complex extension would be to consider the heterogeneity of agents in the model. You could assume that households have different preferences or abilities to save, and that they face different constraints. This would require you to use dynamic programming methods that can handle optimization problems with multiple agents.

**Analyze policy interventions**: Finally, you could use the consumption savings model to analyze the effects of policy interventions on households. For example, you could simulate the effects of a tax cut or a change in interest rates on household consumption and savings decisions. This would allow you to evaluate the effectiveness of different policy interventions in achieving their intended goals.

Overall, extending a consumption savings model in a dynamic programming project requires careful consideration of the features and assumptions of the model and how they can be modified to capture real-world phenomena. It also requires knowledge of dynamic programming methods and how to apply them to solve optimization problems."

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize
#%pip install -e /Users/ElenaCero/Desktop/Polit/9.semester/GEModelTools

import sys
sys.path.append('hank_labor_model')
import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from EconModel import EconModelClass, jit 

## Baseline Consumption Savings Model

In [8]:
from numba import njit
from consav.grids import nonlinspace_jit
import numpy as np

def last_period(m,c,l,inv_v,a_max,Na,m_phi,sigma,nu,phi):
    # last period consume all       
    m[-1,:] = nonlinspace_jit(0,a_max,Na+1,m_phi)
    c[-1,:] = m[-1,:]
    l[-1,:] = np.zeros_like(c)
    for i in range(1, Na+1):
        c_now = c[-1, i]
        l_now = l[-1, i]
        inv_v[-1, i] = 1/((c_now**(1.0-sigma) - 1.0)/(1.0-sigma) - nu*(l_now**(1.0+phi))/(1.0+phi))
    
    inv_v[-1, 0] = 0

    print(m.shape, c.shape, l.shape, inv_v.shape)

    
Na = 100
c = np.zeros((1, Na+1))
l = np.zeros((1, Na+1))
m = np.zeros((1, Na+1))
inv_v = np.zeros((1, Na+1))
a_max = 100.0 # maximum point in grid for a
m_phi = 1.1
sigma = 2.0 # CRRA coefficient
nu= 2.0
phi = 0.9
last_period(m,c,l,inv_v,a_max,Na,m_phi,sigma,nu,phi)

(1, 101) (1, 101) (1, 101) (1, 101)
[[  0.           0.63095734   1.26490353   1.90187924   2.54192611
    3.18508676   3.83140483   4.48092499   5.133693     5.78975575
    6.4491613    7.1119589    7.77819904   8.44793353   9.12121548
    9.79809943  10.47864133  11.16289864  11.85093039  12.5427972
   13.23856137  13.93828696  14.64203985  15.34988777  16.06190047
   16.77814972  17.49870944  18.22365576  18.95306716  19.68702453
   20.42561132  21.1689136   21.91702024  22.67002302  23.42801672
   24.19109935  24.95937223  25.73294022  26.51191182  27.29639943
   28.08651952  28.88239281  29.68414458  30.49190481  31.30580856
   32.12599613  32.95261347  33.78581243  34.62575114  35.4725944
   36.32651406  37.18768947  38.05630798  38.93256544  39.81666674
   40.70882648  41.6092696   42.51823209  43.43596185  44.36271952
   45.29877944  46.24443075  47.1999785   48.16574502  49.14207126
   50.12931848  51.12786995  52.13813298  53.16054112  54.19555672
   55.24367367  56.30542069 

In [3]:
from model import ConSavingLaborModel

#Load baseline settings
model = ConSavingLaborModel(name='baseline')

#Solve and simulate
# model.precompile_numba()
model.solve()
# model.simulate()

par = model.par
sol = model.sol
# sim = model.sim

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Traceback (most recent call last):
  File "/Users/nicolaibernsen/Documents/KU/10.semester/Dynamic Programming/Dynamic-Programming-Project/hank_labor_model/model.py", line 195, in solve
    lastperiod.last_period(par,sol)
ValueError: cannot assign slice from input of different size


ValueError: cannot assign slice from input of different size

In [ ]:
model.solve_hh_ss(do_print=True)

### Value Function Iteration

In [ ]:
model_vfi = ConSavModelClass()

par = model_vfi.par
sol = model_vfi.sol
sim = model_vfi.sim

model_vfi.solve(do_print=True,algo='vfi')

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

I = par.a_grid < 500

# a. consumption
ax = fig.add_subplot(1,3,1)
ax.set_title(f'consumption')

for i_z,z in enumerate(par.z_grid):
    if i_z%3 == 0 or i_z == par.Nz-1:
        ax.plot(par.a_grid[I],sol.c[i_z,:],label=f'z = {z:.2f}')

ax.legend(frameon=True)
ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('consumption, $c_t$')

# b. saving
ax = fig.add_subplot(1,3,2)
ax.set_title(f'saving')

for i_z,z in enumerate(par.z_grid):
    if i_z%3 == 0 or i_z == par.Nz-1:
        ax.plot(par.a_grid[I],sol.a[i_z,:]-par.a_grid[I],label=f'z = {z:.2f}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('savings change, $a_{t}-a_{t-1}$')

# Labor Supply

#fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(1,3,3)

ax.set_title(f'Labor Supply')

for i_z,z in enumerate(par.z_grid):
    if i_z%3 == 0 or i_z == par.Nz-1:
        ax.plot(par.a_grid[I],sol.ell[i_z,:],label=f'z = {z:.2f}')

ax.grid(visible=True, which = 'major', linestyle='-', linewidth=0.5, color='0.9', zorder=0)
ax.tick_params(axis='both', bottom=True, top=True, left=True, right=True, direction='in', which='both')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('Labor supply, $l_t$')

fig.tight_layout()

### Endogenous Grid Method

In [ ]:
model_egm = model_vfi.copy()

In [ ]:
model_egm.solve(do_print=True,algo='egm')

### Compare Solution Methods

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
for i_z in range(par.Nz):
    ax.plot(par.a_grid,model_vfi.sol.a[i_z,:])
    
ax.set_xlabel('$a_{t-1}$');
ax.set_ylabel('diff. to egm');